<a href="https://colab.research.google.com/github/kleinfossil/googlecolabfree_stable_diffusion/blob/easy_sd/Easy_Dreambooth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome
This is a super simplified version to train your AI based on your pictures.

If you just want to create pictures go to my [easy stable diffusion page](https://colab.research.google.com/drive/1spMMW7J7Y-PNNIoJuhyxqHpIQFxcy5Ft?usp=sharing).

More Details on my [detailed AI page](https://colab.research.google.com/drive/1YnAbmwpBuMOKP57RooXwQw01Z9r3Epg_?usp=sharing).

# How to use?
This will train an AI model on pictures you will upload. 

This trained AI model will understand your trigger word.
</br> Means if you train it on pictures of your cat and name the trigger "**mycat**" you can use it in the AI prompt. E.g. "**A picture of mycat**". 

The class prompt is a general category of your trigger word. In this case the class prompt could be "**a picture of a cat**" or just "**cat**"

<font color=green>**INFO:**</font> 
1.   When you press the arrow, you will be asked to upload pictures. These pictures will be **deleted** after you close the runtime.
2.   Only upload **5-20 pictures**
3.   Training an AI takes time. It will take about: **40 Minutes**. Depending on the amout of pictures you upload (more take more time)
4.   Everything runs in a virtual machine on google colab. If you do not click "Save_to_Google_Drive your model will just exists until you close your browser or disconnect this session.
5.   Savining on Google Drive will take **2GB** per model.
6.   Google offers **free** computing power but it is **limited** to a 3-4 hours a day. So if you do not use it close the browser tab or click on: Runtime/Disconnect and delete runtime.

# Trainer Your Model

In [ ]:
# This code was made so that a model can be trained with one click. 
# Thats the reason why I put all the code in one Cell. 
# If you want to split it you can do it at the "---" points

from IPython.display import clear_output

# --- Setup Parameters ---
print("Setup Parameters...")

import re
# Input Parameters
MODEL_NAME = "runwayml/stable-diffusion-v1-5" 

#@markdown Write a short unique word.
Your_Trigger_Word = "kasfaceb" #@param {type:"string"}
#@markdown Write a word or short text which discribes your pictures.
Class_Prompt = "head shot of a man" #@param {type:"string"}
#@markdown Check if you want to save your model. Requires 2GB
Save_to_Google_Drive = True #@param {type:"boolean"}

# This will use a regex to make the prompt a valid foldername
your_prompt_as_folder_name = output = re.sub(r'\W+', '', Your_Trigger_Word)
class_prompt_as_folder_name = output = re.sub(r'\W+', '', Class_Prompt)

#While training the AI will need a directory. 
Instance_Data_Directory = f"/content/data/{your_prompt_as_folder_name}"
Class_Data_Directory = f"/content/data/{class_prompt_as_folder_name}"
OUTPUT_DIR = f"/content/model/{your_prompt_as_folder_name}"


# You can also add multiple concepts here. Try tweaking `--max_train_steps` accordingly.

concepts_list = [
    {
        "instance_prompt":      Your_Trigger_Word,
        "class_prompt":         Class_Prompt,
        "instance_data_dir":    Instance_Data_Directory,
        "class_data_dir":       Class_Data_Directory
    },
#     {
#         "instance_prompt":      "photo of ukj person",
#         "class_prompt":         "photo of a person",
#         "instance_data_dir":    "/content/data/ukj",
#         "class_data_dir":       "/content/data/person"
#     }
]

# This just creates a json of the concepts_list. Maybe one da I want to train more then one concept.
import json
import os
for c in concepts_list:
    os.makedirs(c["instance_data_dir"], exist_ok=True)

with open("concepts_list.json", "w") as f:
    json.dump(concepts_list, f, indent=4)


# --- Connect to Google Drive if required ---
print("Connecting to Google Drive...")
if Save_to_Google_Drive:
    from google.colab import drive
    drive.mount('/content/drive')


# --- Upload Images ---
print("Upload images...")

import os
from google.colab import files
import shutil

for c in concepts_list:
    print(f"Uploading instance images for `{c['instance_prompt']}`")
    uploaded = files.upload()
    for filename in uploaded.keys():
        dst_path = os.path.join(c['instance_data_dir'], filename)
        shutil.move(filename, dst_path)
clear_output()


# --- Install Requirements ---
print("Install requirements...")

# Install Dreambooth requirements
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
clear_output()
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
clear_output()
%pip install git+https://github.com/ShivamShrirao/diffusers
clear_output()
%pip install -U --pre triton
clear_output()
%pip install accelerate==0.12.0 transformers ftfy bitsandbytes gradio natsort
clear_output()

# Install xformers
%pip install -q https://github.com/brian6091/xformers-wheels/releases/download/0.0.15.dev0%2B4c06c79/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl
clear_output()
# These were compiled on Tesla T4.

# If precompiled wheels don't work, install it with the following command. It will take around 40 minutes to compile.
# %pip install git+https://github.com/facebookresearch/xformers@4c06c79#egg=xformers


# --- Train the Model ---
print("Train Model...")

# Count the number of images to get a rough idea of the training steps required.
# A rule of thumb is 100 steps per image.
import os

# folder path
dir_path = Instance_Data_Directory
number_of_files = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        number_of_files += 1

MAX_TRAINING_STEPS = int(number_of_files*100)

# Launch the Training

!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --revision="fp16" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1337 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=50 \
  --sample_batch_size=4 \
  --max_train_steps=$MAX_TRAINING_STEPS \
  --save_interval=30000 \
  --save_sample_prompt=Instance_Prompt \
  --concepts_list="concepts_list.json"

clear_output()

# -- Create Packaged Model ---

CKPT_PATH = f"/content/packaged/{your_prompt_as_folder_name}" + ".ckpt"
GDRIVE_PATH = f"/content/drive/MyDrive/Dreambooth/models/"

#Run conversion.
half_arg = ""

#Whether to convert to fp16, takes half the space (2GB).
WEIGHTS_DIR = ""
if WEIGHTS_DIR == "":
    from natsort import natsorted
    from glob import glob
    import os
    WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
%mkdir /content/packaged

fp16 = True
if fp16:
    half_arg = "--half"
!python convert_diffusers_to_original_stable_diffusion.py --model_path $WEIGHTS_DIR  --checkpoint_path $CKPT_PATH $half_arg
final_model_path = CKPT_PATH
clear_output()
print(f"Path to local Model: {final_model_path}")

# -- Copy Model to Google Drive if required ---
if Save_to_Google_Drive:
  print("Copy Model to Google Drive...")
  # copy source to destination
  %mkdir -p $GDRIVE_PATH
  %cp -av $CKPT_PATH $GDRIVE_PATH
  clear_output()
  print(f"You can find your Model on your Google Drive here: {GDRIVE_PATH}")


# Test Your Model
Let's test your model. 
First time execution will take **3 Minutes**. Afterwards it takes just seconds


In [ ]:
from IPython.display import clear_output # Will clear lines in the text output
import os
from torch import autocast

# --- Select Model Path ---
#@markdown **Model Path**
#@markdown
#@markdown Put the path to your model here. E.g. /content/drive/MyDrive/Dreambooth/models/kasmediumshot.ckpt
#@markdown
#@markdown If you just created a model you can **keep this empty**
Model_Path = "" #@param {type:"string"}
model_path_was_filled = True

# If the training cell was just executed then this data can be used.
if Model_Path == "":
  model_path_was_filled = False 
else:
  if "/drive/MyDrive/" in Model_Path:
    print("Connecting to Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive') 
    clear_output()


# --- Run easy stable diffusion code ---

# Checks if pipe already exists. 
# If yes it will ignore the except block
# If not it will get an NameError and execute the except block
try: pipe
except NameError:
  
  # Install addtional modules
  %pip install diffusers
  clear_output()
  %pip install transformers
  clear_output()
  %pip install OmegaConf
  clear_output()
  %pip install accelerate
  clear_output()

  # conditional import of the just installed modules.
  import torch
  from diffusers import StableDiffusionPipeline, DDIMScheduler
  from omegaconf import OmegaConf  

  if model_path_was_filled:
    # This Code will unpack the ckpt file. This is only needed if a packed model was provided. 
    # Not required if the model was justed created 
    
    # This converts the ckpt back to a normal model. 
    TEMPORARY_MODEL = "/content/unpacked_model"   
    if os.path.exists(TEMPORARY_MODEL):
      print("Model Path already exists")
    else:
      print("Unpack Model...")
      %mkdir $TEMPORARY_MODEL
      #Downloads and execute a file which was written to unpack ckpt files for stable diffusion models
      print("Download Stable Diffusion ckpt converter...")
      !wget -O convert_original_stable_diffusion_to_diffusers.py https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
      clear_output()
      print("Convert ckpt to model...")
      !python /content/convert_original_stable_diffusion_to_diffusers.py --checkpoint_path $Model_Path --dump_path $TEMPORARY_MODEL
      clear_output()
    
    WEIGHTS_DIR = TEMPORARY_MODEL
 
  # Set a specific Scheduler for the AI
  scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
  # Create pipe with model
  clear_output()
  print("Preparing Model Pipe...")
  pipe = StableDiffusionPipeline.from_pretrained(
    WEIGHTS_DIR, 
    scheduler=scheduler, 
    safety_checker=None, 
    torch_dtype=torch.float16
  ).to("cuda")
  clear_output()

#@markdown **Write a prompt with your trigger word**
prompt = "kasfaceb as an astronaut, cinematic, epic, dramatic light, futuristic, cyberpunk, photo realistic, photography, detailed, landscape shot" #@param {type:"string"}
#@markdown **How many Images do you want?** </br> More images take more time (~15seconds per image).
images = 10 #@param {type:"number"}

#Basic input Parameter - Play around with them when you are more advanced
negative_prompt = "" # Add a negative prompt
num_samples = images # Number of images generated
guidance_scale = 7.5 # How close should be the images to the text
num_inference_steps = 50 # Number of steps till the image is generated (50 is recommended)
height = 512 # hight of the output image
width = 512 #@ width of the output image

# Start generation of images
print("Create Images...")
with autocast("cuda"):
    images = pipe(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
    ).images

# Display the images below each other
for img in images:
    display(img)
